In [1]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader
from torchvision import transforms as transforms

%matplotlib inline

In [2]:
from dataset import CocoDataset, display_image_annot, display_bbox_annot

In [3]:
root = 'coco'
img_paths = 'images'
train_dataset = CocoDataset(root, data_type=os.path.join(root, 'train2017'), transforms=None, in_memory=False, is_debug=False)
val_dataset = CocoDataset(root, data_type=os.path.join(root, 'val2017'), transforms=None)

loading annotations into memory...
Done (t=10.22s)
creating index...
index created!
loading annotations into memory...
Done (t=0.25s)
creating index...
index created!


In [4]:
import multiprocessing as mp
import torch.backends.cudnn as cudnn

print(f'Usable threads: { torch.get_num_threads() }')
print(f'cuda version: { torch.version.cuda }\tcudnn version: { cudnn.version() }')

num_workers = (0.4 * mp.cpu_count())
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, pin_memory=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, pin_memory=True, num_workers=num_workers)

Usable threads: 12
cuda version: 11.1	cudnn version: 8005


In [6]:
from nets import YoloNet

'''
    Parameters:
    batch = 24
    epochs = 120
    input_multiple = 32

    Data augmentation:
    * normalized and scaled

    * divide data into SxS (object check if object center falls into thius grid cell), useful for later
    * (x, y) -> center location
    * (w, h)

    Training:
    * Pr(obj) = confidence score and during training: 
        Pr(conf) = Pr(obj) * IoU(bbox_pred, bbox_true)

    * Pr(class_i | obj) and during training: 
        Pr(obj) = regressed
        Pr(class_i | obj) = Pr(class_i) / Pr(obj)
'''

class PersonDetection:
    pass